In [1]:
import os
import pandas as pd
import pickle
from scipy.sparse import csr_matrix, lil_matrix, save_npz
from datetime import datetime
import numpy as np
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import joblib

In [2]:
def load_data(raw_data_relative_path, filename):
    """
    Charge les données des fichiers CSV dans des DataFrames pandas.

    Args:
        raw_data_relative_path (str): Chemin vers le répertoire contenant les fichiers CSV.

    Returns:
        tuple: DataFrames pour les évaluations, les films et les liens.
    """
    try:
        if "ratings" in filename:
            df = pd.read_csv(
                f"{raw_data_relative_path}/{filename}",
                usecols=["userid", "movieid", "rating"],  # Sélectionner les colonnes
                dtype={"rating": "float32", "userid": str, "movieid": str},
            )
            return df
        elif "movies" in filename:
            df = pd.read_csv(
                f"{raw_data_relative_path}/{filename}",
                usecols=["movieid", "title", "genres"],  # Sélectionner les colonnes
                dtype={"movieid": str, "title": str, "genres": str},
            )
            return df
        print(f"Fichier {filename} chargé avec succès.")
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except pd.errors.EmptyDataError as e:
        print(f"No data: {e}")
    except Exception as e:
        print(f"An error occurred while loading data: {e}")


In [3]:
def filterred_data(df):
    """
    Filtrer les données pour ne conserver que les films ayant reçu au moins 5 évaluations
    et les utilsateurs ayant évalués au moins 10 films.
    """
    user_counts = df["userid"].value_counts()
    users_with_more_than_10_ratings = user_counts[user_counts > 10].index

    # Étape 2 : Compter le nombre de notes par film
    movie_counts = df["movieid"].value_counts()
    movies_with_at_least_5_ratings = movie_counts[movie_counts >= 5].index

    # Étape 3 : Filtrer le DataFrame
    df = df[
        (df["userid"].isin(users_with_more_than_10_ratings))
        & (df["movieid"].isin(movies_with_at_least_5_ratings))
    ]

    return df

In [47]:
def train_TFIDF_model(df, data_directory):
    """
    Entraîne un modèle TF-IDF pour extraire des caractéristiques des genres de films.
    """
    # Démarrer une nouvelle expérience MLflow
    # mlflow.start_run()

    start_time = datetime.now()  # Démarrer la mesure du temps

    # Vérifier les colonnes et le contenu
    print("Colonnes du DataFrame :", df.columns)
    print("Aperçu du DataFrame :")
    print(df.head())

    # Créer une instance de TfidfVectorizer
    tfidf = TfidfVectorizer()

    # Calculer la matrice TF-IDF
    tfidf_matrix = tfidf.fit_transform(df["genres"])

    # Afficher la taille de la matrice
    print(f"Dimensions de notre matrice TF-IDF : {tfidf_matrix.shape}")

    # Calculer la similarité cosinus par morceaux
    sim_cosinus = cosine_similarity(tfidf_matrix, tfidf_matrix)
    print(f"Dimensions de la matrice de similarité cosinus : {sim_cosinus.shape}")

    os.makedirs(data_directory, exist_ok=True)  # Crée le répertoire si nécessaire

    # Sauvegarder les éléments essentiels
    joblib.dump(tfidf, os.path.join(data_directory, 'tfidf_model.joblib'))
    joblib.dump(sim_cosinus, os.path.join(data_directory, 'sim_cosinus.joblib'))
    df[['movieid']].to_csv(os.path.join(data_directory, 'movieid.csv'), index=False)

    return tfidf, sim_cosinus, df['movieid']


In [48]:
from tabulate import tabulate

def recommandations(titre, sim_cosinus, movieid, num_recommandations=10):
    """Fonction qui à partir des indices trouvés, renvoie les movie_id des films les plus similaires."""
    # récupérer dans idx l'indice associé au titre depuis la série indices
    #idx = indices[titre]
    idx = movieid.index(titre)
    # garder dans une liste les scores de similarité correspondants à l'index du film cible
    score_sim = list(enumerate(sim_cosinus[idx]))
    #  trier les scores de similarité, trouver les plus similaires et récupérer ses indices
    score_sim = sorted(score_sim, key=lambda x: x[1], reverse=True)
    # Obtenir les scores des 10 films les plus similaires
    top_similair = score_sim[1:num_recommandations+1]
    # Obtenir les indices des films
    res = [(movieid[idx], score) for idx, score in top_similair]
    # Renvoyer les movie_id des films les plus similaires
    return tabulate(res, headers=["movie_id", "Score de similarité"], tablefmt="pretty")

In [72]:
def load_tfidf_model_artifacts(data_directory):
    """Charge les artefacts du modèle TF-IDF sauvegardés."""
    tfidf = joblib.load(os.path.join(data_directory, 'tfidf_model.joblib'))
    sim_cosinus = joblib.load(os.path.join(data_directory, 'sim_cosinus.joblib'))
    movieid = pd.read_csv(os.path.join(data_directory, 'movieid.csv'))['movieid'].tolist()
    return tfidf, sim_cosinus, movieid

In [54]:
my_project_directory = os.path.join("/home/antoine/PROJET_MLOPS_RECO_MOVIES/")
raw_data_relative_path = os.path.join(my_project_directory, "data/raw/silver")
tfidf, sim_cosinus, movieid = load_tfidf_model_artifacts(data_directory)
movies = load_data(raw_data_relative_path, "processed_movies.csv")
ratings = load_data(raw_data_relative_path, "processed_ratings.csv")
df = pd.merge(ratings, movies, on="movieid", how="left")
df = filterred_data(df)

In [26]:
tfidf, sim_cosinus, df['movieid'] = train_TFIDF_model(df, data_directory)

# Faire une recommandation
recommandations_result = recommandations(movieid[0], sim_cosinus, movieid, num_recommandations=10)
print(recommandations_result)


In [50]:
from tabulate import tabulate

def recommandations(titre, sim_cosinus, movies, num_recommandations=10):
    """Fonction qui à partir des indices trouvés, renvoie les movie_id des films les plus similaires."""
    # récupérer dans idx l'indice associé au titre depuis la série indices
    #idx = indices[titre]
    idx = movies.index(titre)
    # garder dans une liste les scores de similarité correspondants à l'index du film cible
    score_sim = list(enumerate(sim_cosinus[idx]))
    #  trier les scores de similarité, trouver les plus similaires et récupérer ses indices
    score_sim = sorted(score_sim, key=lambda x: x[1], reverse=True)
    # Obtenir les scores des 10 films les plus similaires
    top_similair = score_sim[1:num_recommandations+1]
    # Obtenir les indices des films
    res = [ (movies[idx], score) for idx, score in top_similair]
    # Renvoyer les movie_id des films les plus similaires
    return tabulate(res, headers=["movie_id", "Score de similarité"], tablefmt="pretty")

In [28]:
movies.head()

,movieid,title,genres
0,1,Toy Story,"Adventure, Animation, Children, Comedy, Fantasy"
1,2,Jumanji,"Adventure, Children, Fantasy"
2,3,Grumpier Old Men,"Comedy, Romance"
3,4,Waiting to Exhale,"Comedy, Drama, Romance"
4,5,Father of the Bride Part II,Comedy


In [77]:
def train_matrix_factorization_model(df, data_directory):
    """
    Entraîne un modèle de factorisation matricielle pour prédire les évaluations des utilisateurs.
    """
    # Démarrer une nouvelle expérience MLflow
    # mlflow.start_run()

    start_time = datetime.now()  # Démarrer la mesure du temps

    df = df.sample(frac=0.08, random_state=42).reset_index(drop=True)
    mat_ratings = pd.pivot_table(
        data=df, values="rating", columns="title", index="userid"
    )
    mat_ratings = (
        mat_ratings + 1
    )  # On ajoute 1 à toutes les notes pour éviter les problèmes de division par 0
    mat_ratings = mat_ratings.fillna(0)
    sparse_ratings = csr_matrix(mat_ratings)
    user_ids = mat_ratings.index.tolist()
    titles = mat_ratings.columns.tolist()
    # Appliquer la factorisation matricielle
    svd = TruncatedSVD(n_components=100)
    ratings_red = svd.fit_transform(sparse_ratings.T)
    item_similarity = cosine_similarity(ratings_red)
    item_similarity = pd.DataFrame(item_similarity, index=titles, columns=titles)

    # Sauvegarder les éléments essentiels
    joblib.dump(svd, os.path.join(data_directory, 'svd_model.joblib'))
    with open(os.path.join(data_directory, 'titles.pkl'), 'wb') as f:
        pickle.dump(titles, f)
    joblib.dump(item_similarity, os.path.join(data_directory, 'item_similarity.joblib'))
    joblib.dump(mat_ratings, os.path.join(data_directory, 'mat_ratings.joblib'))

    return mat_ratings, item_similarity, titles, svd